In [1]:
import pandas as pd
import numpy as np

In [2]:
train=pd.read_parquet('../Data/train_cleaned.parquet', engine='pyarrow')
val=pd.read_parquet('../Data/val_cleaned.parquet', engine='pyarrow')

In [3]:
val = val.sort_values(['id2', 'id4'])
train=train.sort_values(['id2', 'id4'])

In [4]:
train['time_since_last_seen'] = train.groupby('id2')['id4'].diff().dt.total_seconds()
val['time_since_last_seen'] = val.groupby('id2')['id4'].diff().dt.total_seconds()

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/1354046335.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train['time_since_last_seen'] = train.groupby('id2')['id4'].diff().dt.total_seconds()
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/1354046335.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val['time_since_last_seen'] = val.groupby('id2')['id4'].diff().dt.total_seconds()


In [5]:
train['session_event_count'] = train.groupby('id2').cumcount() + 1
val['session_event_count'] = val.groupby('id2').cumcount() + 1

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/1748974423.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train['session_event_count'] = train.groupby('id2').cumcount() + 1
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/1748974423.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val['session_event_count'] = val.groupby('id2').cumcount() + 1


In [6]:
# 1. Define the mapping for the offer category columns
offer_category_mapping = {
    'f226': 'Business',
    'f227': 'Dining',
    'f228': 'Entertainment',
    'f229': 'Retail',
    'f230': 'Services',
    'f231': 'Shopping',
    'f232': 'Travel'
}

# Get the list of the one-hot encoded columns
ohe_offer_cols = list(offer_category_mapping.keys())

# 2. Process both train and val DataFrames
for df in [train, val]:
    # --- Create the combined string category ---
    # Initialize the new column with a default value
    df['offer_category'] = 'other'
    
    # Create a boolean mask for rows with at least one category
    mask = df[ohe_offer_cols].sum(axis=1) > 0

    # For the rows that have categories, combine them with an underscore
    # The .dot() method is an efficient way to do this matrix multiplication
    df.loc[mask, 'offer_category'] = (
        df.loc[mask, ohe_offer_cols]
        .dot(pd.Series(offer_category_mapping) + '_')
        .str.rstrip('_')
    )

    # --- Create the count of categories ---
    df['num_offer_categories'] = df[ohe_offer_cols].sum(axis=1)

print("New 'offer_category' and 'num_offer_categories' columns created successfully.")

# --- 3. Verify the result ---
print("\nSample of the new columns in the 'train' DataFrame:")
print(train[['offer_category', 'num_offer_categories']].head())

print("\nDistribution of the number of categories per offer:")
print(train['num_offer_categories'].value_counts())

New 'offer_category' and 'num_offer_categories' columns created successfully.

Sample of the new columns in the 'train' DataFrame:
       offer_category  num_offer_categories
228640       Services                     1
46930        Shopping                     1
46929        Shopping                     1
12738        Shopping                     1
12692        Shopping                     1

Distribution of the number of categories per offer:
num_offer_categories
1    539572
2      6604
4      2129
7        14
Name: count, dtype: int64


In [7]:
sub_category_mapping = {

'f233': 'Activities and leisure',

'f234': 'Advertising',

'f235': 'American',

'f236': 'Animal services',

'f237': 'Apparel',

'f238': 'Argentinean',

'f239': 'Asian',

'f240': 'Australian',

'f241': 'Auto fuel',

'f242': 'Auto rental',

'f243': 'Auto sales',

'f244': 'Auto services',

'f245': 'Automotive',

'f246': 'Bar night',

'f247': 'Books music',

'f248': 'British',

'f249': 'Carry out',

'f250': 'Casual',

'f251': 'Chinese',

'f252': 'Clothing',

'f253': 'Coffee desserts',

'f254': 'Communications',

'f255': 'Computer services',

'f256': 'Constructions',

'f257': 'Cruises',

'f258': 'Cuisine',

'f259': 'Delis',

'f260': 'Education',

'f261': 'Electronics',

'f262': 'European',

'f263': 'Finance',

'f264': 'Food store',

'f265': 'Food wine',

'f266': 'French',

'f267': 'Fusion',

'f268': 'German',

'f269': 'Gifts',

'f270': 'Golf',

'f271': 'Greek',

'f272': 'Health care',

'f273': 'Hobbies',

'f274': 'Home furnishing',

'f275': 'Home garden',

'f276': 'Hotel resorts',

'f277': 'Indian',

'f278': 'Italian',

'f279': 'Japanese',

'f280': 'Jewelry',

'f281': 'Mediterranean',

'f282': 'Mexican',

'f283': 'Office products',

'f284': 'Office supplies',

'f285': 'Other',

'f286': 'Other cuisine',

'f287': 'Other services',

'f288': 'Other shopping',

'f289': 'Parking',

'f290': 'Parks zoos',

'f291': 'Personal care',

'f292': 'Printing',

'f293': 'Professional',

'f294': 'Real estate',

'f295': 'Residential care',

'f296': 'Restaurant',

'f297': 'Seafood',

'f298': 'Services',

'f299': 'Shipping services',

'f300': 'Spanish',

'f301': 'Sports recreation',

'f302': 'Steakhouse',

'f303': 'Thai',

'f304': 'Tour',

'f305': 'Transportation',

'f306': 'Travel agencies',

'f307': 'Utilities',

'f308': 'Vegetarian',

'f309': 'Wholesale'

}

In [8]:
# Get the list of the one-hot encoded columns
ohe_cols = list(sub_category_mapping.keys())

# --- For the train DataFrame ---
# Initialize the new column with a default value (e.g., 'None')
train['sub_category'] = 'None'

# Create a boolean mask for rows with at least one category
mask_train = train[ohe_cols].sum(axis=1) > 0

# For the rows that have categories, apply the logic
train.loc[mask_train, 'sub_category'] = (
    train.loc[mask_train, ohe_cols]
    .dot(pd.Series(sub_category_mapping) + '_')
    .str.rstrip('_')
)


# --- For the validation DataFrame ---
val['sub_category'] = 'None'
mask_val = val[ohe_cols].sum(axis=1) > 0
val.loc[mask_val, 'sub_category'] = (
    val.loc[mask_val, ohe_cols]
    .dot(pd.Series(sub_category_mapping) + '_')
    .str.rstrip('_')
)

# (HIGHLY RECOMMENDED) Create a new feature for the number of categories
train['num_sub_categories'] = train[ohe_cols].sum(axis=1)
val['num_sub_categories'] = val[ohe_cols].sum(axis=1)

print("Combined categories created. Example:")
# Show some of the multi-tagged categories you created
print(train[train['num_sub_categories'] > 1]['sub_category'].head())

Combined categories created. Example:
12730    Home furnishing_Home garden
12673           Gifts_Other shopping
12666    Home furnishing_Home garden
12672               Clothing_Jewelry
12668          Gifts_Home furnishing
Name: sub_category, dtype: object


In [9]:
# remove rows with 77 sub_categories
train = train[train['num_sub_categories'] != 77]
val = val[val['num_sub_categories'] != 77]

In [10]:
drop_features = list(offer_category_mapping.keys())
train.drop(drop_features, axis=1, inplace=True)
val.drop(drop_features, axis=1, inplace=True)

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/385752250.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.drop(drop_features, axis=1, inplace=True)


In [11]:
drop_features = list(sub_category_mapping.keys())

In [12]:
train.drop(drop_features, axis=1, inplace=True)
val.drop(drop_features, axis=1, inplace=True)

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/4259240841.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.drop(drop_features, axis=1, inplace=True)


In [13]:
train['previous_offer_category'] = train.groupby('id2')['offer_category'].shift(1)
val['previous_offer_category'] = val.groupby('id2')['offer_category'].shift(1)

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/2647096649.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train['previous_offer_category'] = train.groupby('id2')['offer_category'].shift(1)
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/2647096649.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val['previous_offer_category'] = val.groupby('id2')['offer_category'].shift(1)
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/2647096649.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [14]:
train['previous_suboffer_category'] = train.groupby('id2')['sub_category'].shift(1)
val['previous_suboffer_category'] = val.groupby('id2')['sub_category'].shift(1)

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/3742752096.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train['previous_suboffer_category'] = train.groupby('id2')['sub_category'].shift(1)
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/3742752096.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val['previous_suboffer_category'] = val.groupby('id2')['sub_category'].shift(1)
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/3742752096.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [15]:
train['previous_suboffer_category'].fillna('missing', inplace=True)
train['previous_offer_category'].fillna('missing', inplace=True)

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/3354526865.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['previous_suboffer_category'].fillna('missing', inplace=True)
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/3354526865.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are 

In [16]:
val['previous_suboffer_category'].fillna('missing', inplace=True)
val['previous_offer_category'].fillna('missing', inplace=True)

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/2621411230.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  val['previous_suboffer_category'].fillna('missing', inplace=True)
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/2621411230.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['previous_suboffer_

In [17]:
# change all object type to categorical type
for col in train.select_dtypes(include='object').columns:
    train[col] = train[col].astype('category')

for col in val.select_dtypes(include='object').columns:
    val[col] = val[col].astype('category')

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/1749172736.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val[col] = val[col].astype('category')
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/1749172736.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val[col] = val[col].astype('category')
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/1749172736.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [18]:
train = train.sort_values(['id2', 'id4'])
val = val.sort_values(['id2', 'id4'])

In [19]:
train['no_of_clicks_previously'] = train.groupby('id2')['y'].cumsum().shift(1)
val['no_of_clicks_previously'] = val.groupby('id2')['y'].cumsum().shift(1)

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/391833619.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train['no_of_clicks_previously'] = train.groupby('id2')['y'].cumsum().shift(1)
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/391833619.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val['no_of_clicks_previously'] = val.groupby('id2')['y'].cumsum().shift(1)


In [20]:
train.loc[train['id2'] != train['id2'].shift(1), 'no_of_clicks_previously'] = 0
val.loc[val['id2'] != val['id2'].shift(1), 'no_of_clicks_previously'] = 0

In [21]:
train['no_of_clicks_previously'] = train['no_of_clicks_previously'].astype(int)
val['no_of_clicks_previously'] = val['no_of_clicks_previously'].astype(int)

In [24]:
train['pace_x_ctr'] = train['time_since_last_seen'] * train['f366'] 
train['session_count_x_ctr'] = train['session_event_count'] * train['f366']

In [25]:
train['pace_x_offer_popularity'] = train['time_since_last_seen'] * train['f347']

In [26]:
val['pace_x_offer_popularity'] = val['time_since_last_seen'] * val['f347']
val['pace_x_ctr'] = val['time_since_last_seen'] * val['f366'] 
val['session_count_x_ctr'] = val['session_event_count'] * val['f366']

In [27]:
train = train.sort_values(['id2', 'id4'])

# 1. Create a column that only holds the timestamp when a click occurs
train['click_timestamp'] = train.loc[train['y'] == 1, 'id4']

# 2. Forward-fill this column to carry the last click time forward
# Now every row knows the time of the most recent click up to that point
train['last_click_timestamp'] = train.groupby('id2')['click_timestamp'].ffill()
train['last_click_timestamp'] = train.groupby('id2')['last_click_timestamp'].shift(1)

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/3607711452.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train['last_click_timestamp'] = train.groupby('id2')['click_timestamp'].ffill()
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/3607711452.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train['last_click_timestamp'] = train.groupby('id2')['last_click_timestamp'].shift(1)


In [28]:
train['time_since_last_click'] = (train['id4'] - train['last_click_timestamp']).dt.total_seconds()
train.drop(columns=['click_timestamp', 'last_click_timestamp'], inplace=True)

In [29]:
val = val.sort_values(['id2', 'id4'])

# 1. Create a column that only holds the timestamp when a click occurs
val['click_timestamp'] = val.loc[val['y'] == 1, 'id4']

# 2. Forward-fill this column to carry the last click time forward
# Now every row knows the time of the most recent click up to that point
val['last_click_timestamp'] = val.groupby('id2')['click_timestamp'].ffill()
val['last_click_timestamp'] = val.groupby('id2')['last_click_timestamp'].shift(1)
val['time_since_last_click'] = (val['id4'] - val['last_click_timestamp']).dt.total_seconds()
val.drop(columns=['click_timestamp', 'last_click_timestamp'], inplace=True)

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/94315800.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val['last_click_timestamp'] = val.groupby('id2')['click_timestamp'].ffill()
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_12870/94315800.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val['last_click_timestamp'] = val.groupby('id2')['last_click_timestamp'].shift(1)


In [30]:
train

,id2,id3,id4,id5,y,f1,f2,f5,f6,f7,...,num_offer_categories,sub_category,num_sub_categories,previous_offer_category,previous_suboffer_category,no_of_clicks_previously,pace_x_ctr,session_count_x_ctr,pace_x_offer_popularity,time_since_last_click
228640,1000078,622070,2023-11-02 03:40:44.605,2023-11-02,0,NaN,NaN,NaN,NaN,NaN,...,1,Computer services,1,missing,missing,0,NaN,NaN,NaN,NaN
46930,1000101,69479,2023-11-02 12:54:11.194,2023-11-02,0,NaN,NaN,NaN,NaN,NaN,...,1,Other,1,missing,missing,0,NaN,0.023810,NaN,NaN
46929,1000101,86651,2023-11-02 12:54:46.722,2023-11-02,0,NaN,NaN,NaN,NaN,NaN,...,1,Other,1,Shopping,Other,0,0.845905,0.047619,NaN,NaN
12738,1000125,78680,2023-11-01 21:06:27.987,2023-11-01,0,NaN,26.0,22.0,NaN,NaN,...,1,Clothing,1,missing,missing,0,NaN,0.004184,NaN,NaN
12692,1000125,481197,2023-11-01 21:06:28.369,2023-11-01,0,NaN,26.0,22.0,NaN,NaN,...,1,Other,1,Shopping,Clothing,0,0.001598,0.008368,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456767,1910541,5420674,2023-11-01 07:43:31.173,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,1,Other,1,Shopping,Other,0,0.000000,0.000000,NaN,NaN
456765,1910541,82192,2023-11-01 07:43:31.452,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,1,Clothing,1,Shopping,Other,0,0.000000,0.000000,0.021704,NaN
456769,1910541,285330,2023-11-02 06:37:12.625,2023-11-02,0,NaN,NaN,63.0,65.0,NaN,...,1,None,0,Shopping,Clothing,0,0.000000,0.000000,14364.764227,NaN
63239,1910565,415582,2023-11-01 11:12:58.728,2023-11-01,0,NaN,NaN,24.0,13.0,19.0,...,1,Auto services,1,missing,missing,0,NaN,NaN,NaN,NaN


In [31]:
# save these into parquet files as Big_train_feature_engineered_2
train.to_parquet('../Data/Big_train_feature_engineered_2.parquet')
val.to_parquet('../Data/Big_val_feature_engineered_2.parquet')